sync facebook threads?



In [ ]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

$$.async();
var loginFacebook, listFacebookThreads, readFacebookThread;
importer.import(['selenium cell'])
    .then(runSeleniumCell => runSeleniumCell([
        'log in facebook',
        'list facebook threads',
        'messages from facebook',
    ]))
    .then(r => {
        loginFacebook = r[0];
        listFacebookThreads = r[1];
        readFacebookThread = r[2];
        return loginFacebook();
    })
    .then(threads => {
        try { threads = JSON.parse(fs.readFileSync(project + '/facebook-threads.json')); }
        catch (e) { threads = []; }
    // TODO: compare glob/stat to see which threads need updating
        return importer.runAllPromises(threads.slice(450, 600)
            .map(t => ((resolve) => readFacebookThread(t).then((r) => resolve(r)))));
    })
    // TODO: when threads gets to 100% call listFacebookThreads()
    //.then(() => listFacebookThreads())


collect facebook profiles?



In [ ]:
var {JSDOM} = require('jsdom');
var importer = require('../Core');
var fs = require('fs');
var path = require('path');
var glob = require('glob');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

var parseFacebookFriends = () => {
    return getAllXPath([
        '//a[contains(@href, "friends_tab")]//@href'
    ])
    .then(friends => friends
          .filter((elem, pos, arr) => arr.indexOf(elem) === pos)
          .map(f => f.replace(/\?.*$/ig, '')))
}

var loginFacebook, likeAllPosts, getAllXPath, scrapeFacebookFriends;
var collectFacebookProfiles = () => {
    var fresh;
    return importer.import('selenium cell')
        .then(runSeleniumCell => runSeleniumCell([
            'log in facebook',
            'like all facebook posts',
            'scrape facebook friends',
        ]))
        .then(r => {
            loginFacebook = r[0];
            likeAllPosts = r[1];
            scrapeFacebookFriends = r[2];
            return loginFacebook();
        })
        // TODO: abstract this data collection from JSDOM
        // TODO: only scrape once per day? use last file instead?
        //.then(() => scrapeFacebookFriends())
        .then(() => {
            var friends = glob.sync('**/*friend*', {cwd: project, nodir: true});
            friends.sort((a, b) => 
                new Date(fs.statSync(path.join(project, a)).mtime).getTime()
                    - new Date(fs.statSync(path.join(project, b)).mtime).getTime());
            return fs.readFileSync(path.join(project, friends.pop())).toString();
        })
        .then(doc => {
            // call script to get all Facebook friends
            var dom = new JSDOM(doc);
            return importer.import('all xpath elements',  {
                client: {
                    execute: (func, ...args) => Promise.resolve({
                        value: func.apply(dom.window.document, args)
                    }),
                    addCommand: () => {
                    }
                },
                document: dom.window.document,
                XPathResult: {ORDERED_NODE_ITERATOR_TYPE: 5}
            })
        })
        .then(r => {
            getAllXPath = r;
            return parseFacebookFriends();
        })
        .then(friends => {
            // use glob.sync to make sure every friend is hit at least once in a rotation
            var existingPosts = glob.sync('**/*-posts-*.json', {cwd: project});
            // TODO: way to tell which part of the URL is unique?
            var friendCount = friends.length;
            fresh = friends.filter(profile => {
                const profileId = profile.replace(/^\/|\/$/ig, '').split('/').pop()
                .replace(/[^a-z0-9]/ig, '_');
                // TODO: check for file.stat instead of year?
                return existingPosts.indexOf(profileId
                    + '-posts-' + (new Date()).getFullYear() + '.json') === -1;
            });
            const percent = Math.round((friendCount - fresh.length) / friendCount * 100);
            console.log((friendCount - fresh.length) + ' / ' + friendCount + ' : '
                + percent
                + '%');
            console.log(fresh.slice(0, 10));
            //return percent === 100 ? scrapeFacebookFriends() : [];
        })
        .then(() => {
            return importer.runAllPromises(fresh.slice(0, 10)
                .map(p => (resolve) => likeAllPosts(p, null).then(r => resolve(r))));
        })
};
module.exports = collectFacebookProfiles;


In [ ]:
var importer = require('../Core');
    /*

    https://www.facebook.com/dadsrawesome/videos/1165913990203850/
    https://www.facebook.com/galacticempireofficial/videos/800461353413445/
        */
$$.async();
importer.import('collect facebook profiles')
    .then(collectFacebookProfiles => collectFacebookProfiles())
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


Initializing webdriver on localhost
41 / 475 : 9%
[ 'https://www.facebook.com/brian.daviesjones.3',
  'https://www.facebook.com/megannutrition',
  'https://www.facebook.com/james.cullinan.315',
  'https://www.facebook.com/aritra.kundu.146',
  'https://www.facebook.com/stephen.baier',
  'https://www.facebook.com/dawid.szmyd.90',
  'https://www.facebook.com/bartlett.marilyn',
  'https://www.facebook.com/profile.php',
  'https://www.facebook.com/vitali.kaush',
  'https://www.facebook.com/samcvey' ]
https://www.facebook.com/brian.daviesjones.3/posts/1666747646682569
https://www.facebook.com/brian.daviesjones.3/posts/1666731290017538
https://www.facebook.com/brian.daviesjones.3/posts/1666725856684748
https://www.facebook.com/IranLandofCultureandTradition/photos/a.1632836633665300.1073741830.1551090508506580/1991756927773267/?type=3
https://www.facebook.com/brian.daviesjones.3/posts/1666723673351633
https://www.facebook.com/brian.daviesjones.3/posts/1667685089922158
https://www.facebook.com/

sync facebook contacts with google contacts?



tell a joke on facebook messenger?

run commands from facebook messenger?

